### This script adds mimicsplus parameters to the CTSM parameterfile

- Step 1: Dimension: ndecomp_pools_max is in original file 8 and has to be updated to 10, because MIMICS+ has more decomposition pools. This is done with scripts from Matvey in Matlab (bc didn't know how to do in xarray)
- Step 2: read the updated netCDF file into this script and add the mimicsplus pools

In [31]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime,date
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import os

In [2]:
#Para_original = xr.open_dataset("/home/elisacw/clm50_params.c240105.nc")
#Para_original.to_netcdf("Para.nc")
#Para = xr.open_dataset("Para.nc")

Matlab_para_ext =  xr.open_dataset("/home/elisacw/Documents/Modelling/parameter_data_ctsm_mimicsplus/parameters_updated_clm_mimicsplus/mathlab_matvey/ctsm60_params_vint1.3_ext.nc")
Matlab_para_ext.to_netcdf("Para.nc")
Para = xr.open_dataset("Para.nc")

In [3]:
Para

<xarray.Dataset>
Dimensions:                           (pft: 79, ndecomp_pools_max: 10, segment: 4, variants: 2, ntill_stages_max: 3, ntill_intensities_max: 2)
Coordinates:
    pftname                           (pft) |S40 ...
  * segment                           (segment) |S40 b'sunlit                ...
    variantnames                      (variants) object ...
Dimensions without coordinates: pft, ndecomp_pools_max, variants, ntill_stages_max, ntill_intensities_max
Data variables: (12/424)
    C2_liq_Brun89                     float64 ...
    FUN_fracfixers                    (pft) float64 ...
    a_coef                            float64 ...
    a_exp                             float64 ...
    a_fix                             (pft) float64 ...
    accum_factor                      float64 ...
    ...                                ...
    z0v_cw                            (pft) float64 ...
    zglc                              float64 ...
    bgc_till_decompk_multipliers      (ntill_stages_max, ndecomp_pools_max, ntill_intensities_max) float64 ...
    mimics_till_decompk_multipliers   (ntill_stages_max, ndecomp_pools_max, ntill_intensities_max) float64 ...
    zbedrock                          float64 ...
    zbedrock_sf                       float64 ...
Attributes:
    Conventions:               CF-1.0
    title:                     Vegetation (Plant Function Type or PFT) constants
    NCO:                       netCDF Operators version 5.1.9 (Homepage = htt...
    nco_openmp_thread_number:  1
    history:                   Thu Feb  8 14:13:42 2024: ncatted -a history,g...

Important information:

Some parameters in MIMICS+ only exist inside of functions. You have to find them and trace them back.

Parameters created in the surface file of CLM (valid for MIMICS and MIMICS+):
- fmet      (metabolic fraction of plat litter)
- fclay     (clay fraction of soil)
- T         (soil temperature)

The tuning coefficients aV and aK (from Michaelis Menten kinetics) are separate in MIMICS+, but in MIMICS they are part of:
 - aV part of Vmod
 - aK part of Kmod

! Check if they are the same!





#### MIMICS+ parameters are added to the parameterfile from CTSM 

##### Copy parameters that are in mimics and mimics+, edid them (in case of difference) rename them to "mimicsplus..." and add them to file

EXAMPLE | changing a variable

Step 1: Copy the variable
copied_mimics_fmet = Para['mimics_fmet'].copy()

Step 2: Edit the copied variable
edit_mimics_fmet = copied_mimics_fmet * 1

Step 3: Add the copied and edited variable to the dataset with a different name
Para['mimicsplus_fmet']=edit_mimics_fmet

Step 4: Validate variable
Para["mimicsplus_fmet"]

In [4]:
# Soil temperature
copied_mimics_t_soi_ref = Para['mimics_t_soi_ref'].copy(deep=True)       # mimics  25
edit_mimics_t_soi_ref = copied_mimics_t_soi_ref                          # mimics+ 25
Para['mimicsplus_t_soi_ref'] = edit_mimics_t_soi_ref
Para["mimicsplus_t_soi_ref"]


# Michaelis menten parameters

# Regression coefficient Vslope (if more fluxes might change)
copied_mimics_vslope = Para['mimics_vslope'].copy(deep=True)             # mimics  0.063
edit_mimics_vslope = copied_mimics_vslope                                # mimics+ 0.063 | Alaska: 0.055 !LITm, LITs, SOMa entering SAPb, LITm, LITs, SOMa entering SAPf
Para['mimicsplus_vslope'] = edit_mimics_vslope
Para["mimicsplus_vslope"]

# Regression intercept Vint
copied_mimics_vint = Para['mimics_vint'].copy(deep=True)                 # mimics  5.47
edit_mimics_vint = copied_mimics_vint                                    # mimics+ 5.47 | Alaska value:585    !LITm, LITs, SOMa entering SAPb, LITm, LITs, SOMa entering SAPf
Para['mimicsplus_vint'] = edit_mimics_vint   
Para["mimicsplus_vint"]

# Regression coefficient Vmod 
copied_mimics_vmod = Para['mimics_vmod'].copy(deep=True)                 # mimics  1.25e-07, 2.50e-08, 1.25e-07, 3.75e-08, 3.75e-08, 2.50e-08
#edit_mimics_vmod = [10.0, 3.0, 10.0, 3.0,5.0, 2.0]                      # mimics+ 10.0, 3.0, 10.0, 3.0,5.0, 2.0
edit_mimics_vmod = [0.000000125, 0.0000000375, 0.000000125, 0.0000000375, 0.0000000625, 0.000000025]
indices_to_change_mimics_vmod =[0,1,2,3,4,5]
copied_mimics_vmod[indices_to_change_mimics_vmod]=edit_mimics_vmod
Para['mimicsplus_vmod'] = copied_mimics_vmod
Para["mimicsplus_vmod"]

# Regression coefficient Kslope 
copied_mimics_kslope = Para['mimics_kslope'].copy(deep=True)             # mimics  0.02, 0.02, 0.02, 0.02, 0.02, 0.02
edit_mimics_kslope = [0.017,0.027,0.017,0.017,0.027,0.017]               # mimics+ 0.017,0.027,0.017,0.017,0.027,0.017
indices_to_change_mimics_kslope =[0,1,2,3,4,5]
copied_mimics_kslope[indices_to_change_mimics_kslope]=edit_mimics_kslope
Para['mimicsplus_kslope'] = copied_mimics_kslope
Para["mimicsplus_kslope"]

# Regression intercept Kint
copied_mimics_kint = Para['mimics_kint'].copy(deep=True)                 # mimics  3.19 
edit_mimics_kint = copied_mimics_kint                                    # mimics+ 3.19 | Alaska value:2.88 !LITm, LITs, SOMa entering SAPb, LITm, LITs, SOMa entering SAPf
Para['mimicsplus_kint'] = edit_mimics_kint
Para["mimicsplus_kint"]

# Kmod
copied_mimics_kmod = Para['mimics_kmod'].copy(deep=True)                 # mimics  0.001953, 0.007812, 0.003906, 0.007812, 0.003906, 0.002604, 0.      ,0.
edit_mimics_kmod = copied_mimics_kmod                                    # mimics+ for now the same, but there is sth going on with P, physical protection scalar
Para['mimicsplus_kmod'] = edit_mimics_kmod
Para["mimicsplus_kmod"]

# mimics_p_scalar = physical protection scalar | NOT SURE
copied_mimics_p_scalar = Para['mimics_p_scalar'].copy(deep=True)         # mimics 0.8, -3
edit_mimics_p_scalar = copied_mimics_p_scalar                            # mimics+ 
Para['mimicsplus_p_scalar'] = edit_mimics_p_scalar
Para["mimicsplus_p_scalar"]

# Change Increase Km from 4 to 6
copied_mimics_ko_r = Para['mimics_ko_r'].copy(deep=True)                 # mimics  4
edit_mimics_ko_r = copied_mimics_ko_r +2                                 # mimics+ 6 | Increases Km (the half saturation constant for oxidation of chemically protected SOM, SOM_c) from mimics
Para['mimicsplus_ko_r'] = edit_mimics_ko_r
Para["mimicsplus_ko_r"]

# Change Increase Km from 4 to 6
copied_mimics_ko_k = Para['mimics_ko_k'].copy(deep=True)                 # mimics  4
edit_mimics_ko_k = copied_mimics_ko_k +2                                 # mimics+ 6 | Increases Km (the half saturation constant for oxidation of chemically protected SOM, SOM_c) from mimics
Para['mimicsplus_ko_k'] = edit_mimics_ko_k
Para["mimicsplus_ko_k"]


# C:N ratio

# Change CN ration fungi
copied_mimics_mimics_cn_k = Para['mimics_cn_k'].copy(deep=True)          # mimics  10 
edit_mimics_mimics_cn_k = copied_mimics_mimics_cn_k - 2                  # mimics+ 8
Para['mimicsplus_cn_k'] = edit_mimics_mimics_cn_k
Para["mimicsplus_cn_k"]

# Change CN ration bacteria 
copied_mimics_cn_k = Para['mimics_cn_r'].copy(deep=True)                 # mimics  6
edit_mimics_cn_k = copied_mimics_cn_k - 1                                # mimics+ 5
Para['mimicsplus_cn_r'] = edit_mimics_cn_k
Para["mimicsplus_cn_r"]

# Change NUE of sapotrophs
copied_mimics_nue_into_mic = Para['mimics_nue_into_mic'].copy(deep=True) # mimics  0.85
edit_mimics_nue_into_mic = copied_mimics_nue_into_mic -0.05              # mimics+ 0.80
Para['mimicsplus_nue_into_mic'] = edit_mimics_nue_into_mic
Para["mimicsplus_nue_into_mic"]


# Fraction of SAP necromass going into SOM pools

# mimics_fphys_r equals SAPb -> SOMp 
copied_mimics_fphys_r = Para['mimics_fphys_r'].copy(deep=True)          # mimics  0.03, 1.3
edit_mimics_fphys_r = copied_mimics_fphys_r                             # mimics+ 0.3 x exp (1.3 x fclay) | weird that difference is so big
Para['mimicsplus_fphys_r'] = edit_mimics_fphys_r
Para["mimicsplus_fphys_r"]

# mimics_fphys_k equals SAPf -> SOMp
copied_mimics_fphys_k = Para['mimics_fphys_k'].copy(deep=True)          # mimics  0.02, 0.8
edit_mimics_fphys_k = copied_mimics_fphys_k                             # mimics+ 0.2 x exp (0.8 x fclay) | weird that difference is so big
Para['mimicsplus_fphys_k'] = edit_mimics_fphys_k
Para["mimicsplus_fphys_k"]

# mimics_fchem_r equals SAPb -> SOMc
copied_mimics_fchem_r = Para['mimics_fchem_r'].copy(deep=True)          # mimics  0.1, -3. ,  3. ,  0. 
edit_mimics_fchem_r = copied_mimics_fchem_r                             # mimics+ 0.1 x exp (-3 x fmet) 
Para['mimicsplus_fchem_r'] = edit_mimics_fchem_r
Para["mimicsplus_fchem_r"]

# mimics_fchem_k equals SAPf -> SOMc
copied_mimics_fchem_k = Para['mimics_fchem_k'].copy(deep=True)          # mimics  0.3, -3. ,  3. ,  0.
edit_mimics_fchem_k = copied_mimics_fchem_k                             # mimics+ 0.3 x exp (-3 x fmet) 
Para['mimicsplus_fchem_k'] = edit_mimics_fchem_k
Para["mimicsplus_fchem_k"]

# mimics_tau_r equals SAPb -> SOMa (equation)
copied_mimics_tau_r = Para['mimics_tau_r'].copy(deep=True)              # mimics  0.00052, 0.3 
edit_mimics_tau_r = copied_mimics_tau_r                                 # mimics+ 1-(fSAPb, SOMp + fSAPb,) 
Para['mimicsplus_tau_r'] = edit_mimics_tau_r
Para["mimicsplus_tau_r"]

# mimics_tau_k equals SAPf -> SOMa (equation)
copied_mimics_tau_k = Para['mimics_tau_k'].copy(deep=True)              # mimics  0.00024, 0.1
edit_mimics_tau_k = copied_mimics_tau_k                                 # mimics+ 1-(fSAPf, SOMp + fSAPf, SOMc) 
Para['mimicsplus_tau_k'] = edit_mimics_tau_k    
Para["mimicsplus_tau_k"]

# Growth efficiency of bacteria and fungi
copied_mimics_mge = Para['mimics_mge'].copy(deep=True)                  # mimics  0.5 , 0.25, 0.5 , 0.7 , 0.35, 0.7 , 0.  , 0.
edit_mimics_mge = copied_mimics_mge                                     # mimics+ CUEb: 0-0.4 | CUEf: 0-0.7 NOT SURE
Para['mimicsplus_mge'] = edit_mimics_mge    
Para["mimicsplus_mge"]


<xarray.DataArray 'mimicsplus_mge' (ndecomp_pools_max: 10)>
array([0.5 , 0.25, 0.5 , 0.7 , 0.35, 0.7 , 0.  , 0.  , 0.  , 0.  ])
Dimensions without coordinates: ndecomp_pools_max
Attributes:
    long_name:  Microbial growth efficiency for each transition into microbia...
    units:      mgC/mgC

In [5]:

# Initial Carbon stocks need vaules (can't be 0) so they are 1 for the added two positions (for now)

# Carbon stocks 
copied_mimics_initial_Cstocks = Para['mimics_initial_Cstocks'].copy(deep=True)             
edit_mimics_initial_Cstocks = [1,1,200,200,200,1,1,1,1,1]
indices_to_change_copied_mimics_initial_Cstocks =[0,1,2,3,4,5,6,7,8,9]
copied_mimics_initial_Cstocks[indices_to_change_copied_mimics_initial_Cstocks]=edit_mimics_initial_Cstocks
Para['mimicsplus_initial_Cstocks'] = copied_mimics_initial_Cstocks
Para["mimicsplus_initial_Cstocks"]


copied_mimics_initial_Cstocks_depth = Para['mimics_initial_Cstocks_depth'].copy(deep=True)    # mimics  1.5
edit_mimics_initial_Cstocks_depth = copied_mimics_initial_Cstocks_depth                       # mimics+ 1.5
Para['mimicsplus_initial_Cstocks_depth'] = edit_mimics_initial_Cstocks_depth
Para["mimicsplus_initial_Cstocks_depth"]




<xarray.DataArray 'mimicsplus_initial_Cstocks_depth' ()>
array(1.5)
Attributes:
    long_name:  Soil depth for initial C stocks for a cold start w MIMICS
    units:      m

In [6]:
# Litter to SOM

#mimics_cn_mod_num # 0.4
copied_mimics_cn_mod_num = Para['mimics_cn_mod_num'].copy(deep=True)           # mimics 0.4
edit_mimics_cn_mod_num = copied_mimics_cn_mod_num                              # mimics+ 
Para['mimicsplus_cn_mod_num'] = edit_mimics_cn_mod_num
Para["mimicsplus_cn_mod_num"]

# is this the stuff that says how much is avaliable from SOMa ? | does this even exist in mIMICS+?
#elin uses pmod instead

# mimics_tau_mod_factor  0 0.1 | is that the "exp" in 0.3 x exp (-3 x fmet) | does not exist in mimics+
copied_mimics_tau_mod_factor = Para['mimics_tau_mod_factor'].copy(deep=True)    # mimics 1
edit_mimics_tau_mod_factor = copied_mimics_tau_mod_factor                       # mimics+ 
Para['mimicsplus_tau_mod_factor'] = edit_mimics_tau_mod_factor 
Para["mimicsplus_tau_mod_factor"]


# mimics_tau_mod_max # 1.3
copied_mimics_tau_mod_max = Para['mimics_tau_mod_max'].copy(deep=True)          # mimics 1.3
edit_mimics_tau_mod_max = copied_mimics_tau_mod_max                             # mimics+ 
Para['mimicsplus_tau_mod_max'] = edit_mimics_tau_mod_max 
Para["mimicsplus_tau_mod_max"]


#mimics_tau_mod_min # 0.6
copied_mimics_tau_mod_min = Para['mimics_tau_mod_min'].copy(deep=True)           # mimics 0.6
edit_mimics_tau_mod_min = copied_mimics_tau_mod_min                              # mimics+ 
Para['mimicsplus_tau_mod_min'] = edit_mimics_tau_mod_min 
Para["mimicsplus_tau_mod_min"]


<xarray.DataArray 'mimicsplus_tau_mod_min' ()>
array(1.5)
Attributes:
    long_name:  minimum modification to microbial turnover rates
    units:      unitless

##### Create parameters new from scratch that are new in mimics+ and add them to file 

In [7]:
# Adding new Mycorrhizal parameters to parameter file

# Mycorrhizal turnover rate (k myc,som)
new_attrs = {'long_name': 'Turnover rate for Mycorrhiza', 'units': 'h⁻1'}
mimicsplus_k_myc_som = xr.DataArray(
    0.000114,
    attrs=new_attrs
)
mimicsplus_k_myc_som
Para["mimicsplus_k_myc_som"] = mimicsplus_k_myc_som

In [8]:
# Mycorrhizal decay rate K_MO
new_attrs = {'long_name': 'Mycorrhizal decay rate', 'units': 'm²gC⁻¹hr⁻¹'}
mimicsplus_k_mo = xr.DataArray(
    0.000000342,
    attrs=new_attrs
)
mimicsplus_k_mo
Para["mimicsplus_k_mo"] = mimicsplus_k_mo


In [9]:
# Maximum mycorrhizal uptake up inorganic nitrogen
new_attrs = {'long_name': 'Max. mycorrhizal uptake up inorg. N', 'units': 'g x g⁻¹h⁻¹'}
mimicsplus_vmax_myc = xr.DataArray(
    0.000205,
    attrs=new_attrs
)
mimicsplus_vmax_myc
Para['mimicsplus_vmax_myc'] = mimicsplus_vmax_myc


In [10]:
# Half saturation constant of ectomycorrhizal uptake of inorg N
new_attrs = {'long_name': 'Half saturation constant of ectomycorrhizal uptake of inorg N', 'units': 'gNm⁻²'}
mimicsplus_k_m_emyc = xr.DataArray(
    0.08,
    attrs=new_attrs
)
mimicsplus_k_m_emyc
Para['mimicsplus_k_m_emyc'] = mimicsplus_k_m_emyc

In [11]:
# Growth efficiency of EcM mycorrhiza 
new_attrs = {'long_name': 'Microbial growth efficiency (mg/mg) for ectomycorrhiza ', 'units': 'unitless'}
mimicsplus_mge_ecm = xr.DataArray(
    0.5,
    attrs=new_attrs
)
mimicsplus_mge_ecm
Para['mimicsplus_mge_ecm'] = mimicsplus_mge_ecm

In [12]:
# Growth efficiency of AM Mycorrhiza 
new_attrs = {'long_name': 'Growth efficiency of arbuscular mycorrhiza ', 'units': 'unitless'}
mimicsplus_mge_am = xr.DataArray(
    0.5,
    attrs=new_attrs
)
mimicsplus_mge_am
Para['mimicsplus_mge_am'] = mimicsplus_mge_am

In [13]:
# Mycorrhizal modifier
new_attrs = {'long_name': 'Mycorrhizal modifier', 'units': 'unitless'}
mimicsplus_r_myc = xr.DataArray(
    1,
    attrs=new_attrs
)
mimicsplus_r_myc
Para['mimicsplus_r_myc'] = mimicsplus_r_myc

In [14]:
# Fraction of mycorrhizal necromass into soil organic matter pools: ----------------------------------------------------------------

# Fraction ectomycorrhizal necromass into physically protected soil organic matter pool
new_attrs = {'long_name': 'Fraction ectomycorrhizal necromass into physically protected soil organic matter pool', 'units': 'unitless'}
mimicsplus_fphys_ecm = xr.DataArray(
    0.4,
    attrs=new_attrs
)
mimicsplus_fphys_ecm
Para['mimicsplus_fphys_ecm'] = mimicsplus_fphys_ecm


In [15]:
# Fraction ectomycorrhizal necromass into chemically protected soil organic matter pool
new_attrs = {'long_name': 'Fraction ectomycorrhizal necromass into chemically protected soil organic matter pool', 'units': 'unitless'}
mimicsplus_fchem_ecm = xr.DataArray(
    0.2,
    attrs=new_attrs
)
mimicsplus_fchem_ecm
Para['mimicsplus_fchem_ecm'] = mimicsplus_fchem_ecm


In [16]:
# Fraction ectomycorrhizal necromass into avaliable soil organic matter pool
new_attrs = {'long_name': 'Fraction ectomycorrhizal necromass into avaliable soil organic matter pool', 'units': 'unitless'}
mimicsplus_tau_ecm = xr.DataArray(
    0.4,
    attrs=new_attrs
)
mimicsplus_tau_ecm
Para['mimicsplus_tau_ecm'] = mimicsplus_tau_ecm


In [17]:
# Fraction arbuscular mycorrhizal necromass into physically protected soil organic matter pool
new_attrs = {'long_name': 'Fraction arbuscular mycorrhizal necromass into physically protected soil organic matter pool', 'units': 'unitless'}
mimicsplus_fphys_am = xr.DataArray(
    0.3,
    attrs=new_attrs
)
mimicsplus_fphys_am
Para['mimicsplus_fphys_am'] = mimicsplus_fphys_am

In [18]:
# Fraction arbuscular mycorrhizal necromass into chemically protected soil organic matter pool
new_attrs = {'long_name': 'Fraction arbuscular mycorrhizal necromass into chemically protected soil organic matter pool', 'units': 'unitless'}
mimicsplus_fchem_am = xr.DataArray(
    0.4,
    attrs=new_attrs
)
mimicsplus_fchem_am
Para['mimicsplus_fchem_am'] = mimicsplus_fchem_am

In [19]:
# Fraction arbuscular mycorrhizal necromass into avaliable soil organic matter pool
new_attrs = {'long_name': 'Fraction arbuscular mycorrhizal necromass into avaliable soil organic matter pool', 'units': 'unitless'}
mimicsplus_tau_am = xr.DataArray(
    0.3,
    attrs=new_attrs
)
mimicsplus_tau_am
Para['mimicsplus_tau_am'] = mimicsplus_tau_am

In [20]:
# Optimal CN ratio for mycorrhizal fungi
new_attrs = {'long_name': 'Optimal CN ratio for mycorrhizal fungi', 'units': 'unitless'}
mimicsplus_cn_myc= xr.DataArray(
    20,
    attrs=new_attrs
)
mimicsplus_cn_myc
Para['mimicsplus_cn_myc'] = mimicsplus_cn_myc


In [21]:
# Fraction of C from vegetation to EcM, that goes into SOMa for mining
new_attrs = {'long_name': 'Mining fraction of C from vegetation to EcM, that goes into SOMa', 'units': 'unitless'}
mimicsplus_fenz= xr.DataArray(
    0.1,
    attrs=new_attrs
)
mimicsplus_fenz
Para['mimicsplus_fenz'] = mimicsplus_fenz


In [22]:
# Fraction of C from vegetation to EcM, that goes into SOMa for mining
new_attrs = {'long_name': 'Fraction of mycorrhizal N uptake that needs to stay within the fungi (not given to plant)', 'units': 'unitless'}
mimicsplus_fgrowth= xr.DataArray(
    0.5,
    attrs=new_attrs
)
mimicsplus_fgrowth
Para['mimicsplus_fgrowth'] = mimicsplus_fgrowth


In [23]:
# Adding parameters from surface data file:
# Those parameters are already in mimics, but the copy & edidt doesn't work for them

# densdep
new_attrs = {'long_name': '', 'units': ''}
mimicsplus_densdep= xr.DataArray(
    1,
    attrs=new_attrs
)
mimicsplus_densdep
Para['mimicsplus_densdep'] = mimicsplus_densdep

In [24]:
# desorpQ10
new_attrs = {'long_name': '', 'units': ''}
mimicsplus_desorpQ10= xr.DataArray(
    1,
    attrs=new_attrs
)
mimicsplus_desorpQ10
Para['mimicsplus_desorpQ10'] = mimicsplus_desorpQ10

In [25]:
# desorp as variant

#data_mimicsplus_desorp = [1.05e-06, -2.00e+00]
data_mimicsplus_desorp = [2e-06, -4.5e+00]

new_attrs = {'long_name': '', 'units': ''}
mimicsplus_desorp= xr.DataArray(
    data= data_mimicsplus_desorp,
    coords={'variant': [0, 1]},
    dims=['variant'],
    attrs=new_attrs
)
mimicsplus_desorp
Para['mimicsplus_desorp'] = mimicsplus_desorp
Para['mimicsplus_desorp']

<xarray.DataArray 'mimicsplus_desorp' (variant: 2)>
array([ 2.0e-06, -4.5e+00])
Coordinates:
  * variant  (variant) int64 0 1
Attributes:
    long_name:  
    units:

In [26]:
# fmet as segment
#mimicsplus_fmet_data = [0.75, 0.85, 0.013, 40.00]
mimicsplusfmet_data = np.array([0.75, 0.85, 0.013, 40.00])
segment_coords = np.array(['sunlit', 'shaded', 'xylem', 'root'])

new_attrs = {'long_name': '', 'units': ''}
mimicsplusfmet = xr.DataArray(
    data= mimicsplusfmet_data,
    coords={'segment': Para['segment'].data},
    dims=['segment'],
    attrs=new_attrs
)

#mimicsplus_fmet
Para['mimicsplus_fmet'] = mimicsplusfmet

#Para['mimicsplus_fmet']

In [29]:
Para.to_netcdf('clm_params_mimicsplus_netcdf4.nc')

In [30]:
para1=xr.open_dataset("clm_params_mimicsplus_netcdf4.nc")
para1

<xarray.Dataset>
Dimensions:                           (pft: 79, ndecomp_pools_max: 10, segment: 4, variants: 2, ntill_stages_max: 3, ntill_intensities_max: 2, variant: 2)
Coordinates:
    pftname                           (pft) |S40 ...
  * segment                           (segment) |S40 b'sunlit                ...
    variantnames                      (variants) object ...
  * variant                           (variant) int64 0 1
Dimensions without coordinates: pft, ndecomp_pools_max, variants, ntill_stages_max, ntill_intensities_max
Data variables: (12/470)
    C2_liq_Brun89                     float64 ...
    FUN_fracfixers                    (pft) float64 ...
    a_coef                            float64 ...
    a_exp                             float64 ...
    a_fix                             (pft) float64 ...
    accum_factor                      float64 ...
    ...                                ...
    mimicsplus_fenz                   float64 ...
    mimicsplus_fgrowth                float64 ...
    mimicsplus_densdep                int64 ...
    mimicsplus_desorpQ10              int64 ...
    mimicsplus_desorp                 (variant) float64 ...
    mimicsplus_fmet                   (segment) float64 ...
Attributes:
    Conventions:               CF-1.0
    title:                     Vegetation (Plant Function Type or PFT) constants
    NCO:                       netCDF Operators version 5.1.9 (Homepage = htt...
    nco_openmp_thread_number:  1
    history:                   Thu Feb  8 14:13:42 2024: ncatted -a history,g...